# Topic Modeling of Data Science Job Postings and Northeastern CS Courses 

In [6]:
#imports 

import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from summa import keywords
import matplotlib as plt
from wordcloud import WordCloud, STOPWORDS
import sys, os

In [2]:
#load data from csv
df = pd.read_csv('data_scientist_united_states_job_postings_jobspikr.csv', low_memory = True)

### EDA and Data Wrangling

In [3]:
#drop not needed columns
drop_cols = ['crawl_timestamp', 'salary_offered', 'job_board','geo', 'cursor', 'contact_email', 'contact_phone_number', 'html_job_description', 'inferred_city', 'inferred_state', 'inferred_country']
df = df.drop(columns  = drop_cols)

#drop all rows with any null columns
df = df.dropna(axis = 0, how = 'any', subset = None, inplace = False)


In [4]:
#change the types of the columns from object to necessary type 
df['category'] = df['category'].astype('category')
df['company_name'] = df['company_name'].astype('string')
df['city'] = df['city'].astype('string')
df['state'] = df['state'].astype('string')
df['country'] = df['country'].astype('string')
df['job_type'] = df['job_type'].astype('category')
df['job_description'] = df['job_description'].astype('string')

- We can see below that the types have been changed from object to the necessary types, and all null values have removed and the data in each row is uniform 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8700 entries, 0 to 9992
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   url              8700 non-null   object  
 1   job_title        8700 non-null   object  
 2   category         8700 non-null   category
 3   company_name     8700 non-null   string  
 4   city             8700 non-null   string  
 5   state            8700 non-null   string  
 6   country          8700 non-null   string  
 7   post_date        8700 non-null   object  
 8   job_description  8700 non-null   string  
 9   job_type         8700 non-null   category
 10  uniq_id          8700 non-null   object  
dtypes: category(2), object(4), string(5)
memory usage: 716.2+ KB


In [6]:
df.head(10)

,url,job_title,category,company_name,city,state,country,post_date,job_description,job_type,uniq_id
0,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,2019-02-06,Read what people are saying about working here...,Undefined,3b6c6acfcba6135a31c83bd7ea493b18
3,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,2019-02-06,Read what people are saying about working here...,Undefined,1c8541cd2c2c924f9391c7d3f526f64e
4,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,2019-02-05,We are seeking an extraordinary Data Scientist...,Full Time,445652a560a5441060857853cf267470
5,https://www.indeed.com/viewjob?jk=14fa3772cd5f...,CIB – Fixed Income Research – Machine Learning...,Accounting/Finance,JP Morgan Chase,New York,NY 10179 (Midtown area),Usa,2019-02-05,Read what people are saying about working here...,Undefined,9571ec617ba209fd9a4f842973a4e9c8
6,https://www.indeed.com/viewjob?jk=c6db96b37f8a...,"Data Scientist, Licensing Operations",Accounting/Finance,Spotify,New York,NY 10011 (Chelsea area),Usa,2019-02-06,Read what people are saying about working here...,Undefined,0ec629c03f3e82651711f2626c23cadb
8,https://www.indeed.com/viewjob?jk=0fc298b9f3a8...,"Data Scientist, Aladdin Wealth Tech, Associate",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,2019-02-06,Read what people are saying about working here...,Undefined,80d64b46bc7c89602f63daf06b9f1b4c
10,https://www.indeed.com/viewjob?jk=fe663233109e...,Data Scientist,Computer/Internet,Northrop Grumman,Monterey,CA 93940,Usa,2019-02-06,Read what people are saying about working here...,Undefined,9cd3ed78e5cac9e516ea41173de2c25f
12,https://www.indeed.com/viewjob?jk=27a8eb5fa40b...,ETL Developer / Data Scientist,Computer/Internet,Noblis,Reston,VA 20191,Usa,2019-02-06,Read what people are saying about working here...,Undefined,53224da901548e7137bbb163d456ba6a
13,https://www.indeed.com/viewjob?jk=5d05c5d90a50...,Research Data Scientist,Computer/Internet,ARUP Laboratories,Salt Lake City,UT,Usa,2019-02-06,Read what people are saying about working here...,Undefined,010b5d671896d26eba50948c7c337f94
18,https://www.indeed.com/viewjob?jk=be9a799359df...,Data Scientist,Computer/Internet,Bank of America,Seattle,WA 98104 (First Hill area),Usa,2019-02-06,Read what people are saying about working here...,Undefined,4a8b875d1acc4f560716561d699aa022


In [7]:
#Create word cloud here: 
!pip install wordcloud
import matplotlib as plt
from wordcloud import WordCloud, STOPWORDS
import sys, os 
os.chdir(sys.path[0])

# Read text 
text = open('data_scientist_united_states_job_postings_jobspikr.csv', mode='r', encoding='utf-8').read()
stopwords = STOPWORDS

wc = WordCloud(
    background_color='white',
    stopwords=stopwords,
    height=200,
    width=400
)

wc.generate(text)

#store to file 
wc.to_file('wordcloud_output.png')

In [8]:
## updates 

In [9]:
corpus = df['job_description']

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
corpus = df['job_description']
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(corpus)

In [11]:
import nltk 
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import re

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

stop_words.append(['make', 'etc', 'made', 'preferred', 'please'])
lemmatizer = nltk.WordNetLemmatizer()

#Function to pre-process the text information

def normalize_document(doc):
    
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A) #re.I (ignore case), re.A (ASCII-only matching)
    doc = doc.lower()
    doc = doc.strip()
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatise document from filtered tokens
    lem_text = [lemmatizer.lemmatize(i) for i in filtered_tokens]
    
    # Remove words that are only one character.
    lem_text = [token for token in lem_text if len(token) > 1]
    
    # Remove numbers, but not words that contain numbers.
    lem_text = [token for token in lem_text if not token.isnumeric()]
    
    doc = ' '.join(lem_text)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(corpus)

In [12]:
from nltk.tokenize import word_tokenize
import gensim.corpora as corpora
# nltk.download('punkt')

#Tokenise the corpus
tokenized_corp = [word_tokenize(i) for i in norm_corpus]

# Create Dictionary
id2word = corpora.Dictionary(tokenized_corp)

#Remove words that don't feature 20 times and those that feature in over 50% of documents
id2word.filter_extremes(no_below=1) #no_above

texts = tokenized_corp

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
import gensim
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=123,
                                       chunksize=100,
                                       passes=100,
                                       per_word_topics=True)

In [14]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"customer" + 0.007*"insight" + 0.006*"modeling" + 0.005*"analytical" '
  '+ 0.005*"algorithm" + 0.005*"advanced" + 0.005*"research" + 0.005*"design" '
  '+ 0.005*"etc" + 0.005*"technical"'),
 (1,
  '0.018*"aws" + 0.010*"client" + 0.009*"please" + 0.008*"service" + '
  '0.008*"candidate" + 0.008*"big" + 0.008*"location" + 0.007*"role" + '
  '0.007*"azure" + 0.006*"customer"'),
 (2,
  '0.008*"client" + 0.007*"status" + 0.007*"employment" + 0.007*"required" + '
  '0.006*"protected" + 0.006*"applicant" + 0.006*"location" + 0.005*"national" '
  '+ 0.005*"industry" + 0.005*"world"'),
 (3,
  '0.012*"--" + 0.011*"set" + 0.008*"algorithm" + 0.008*"customer" + '
  '0.008*"etc" + 0.007*"drive" + 0.007*"qualification" + 0.007*"large" + '
  '0.006*"insight" + 0.006*"support"'),
 (4,
  '0.013*"use" + 0.012*"training" + 0.010*"set" + 0.010*"access" + '
  '0.010*"research" + 0.009*"youll" + 0.008*"technical" + '
  '0.007*"certification" + 0.007*"world" + 0.007*"change"'),
 (5,
  '0.008*"

In [15]:
# - TODO: 
# - remove stop words we have identified 
# - coherence score to determine optimal number of topics
# - score the model, accuracy mean_score,

In [16]:
lda_dict = {0:'Financial Risk & Customer Management', 
            1:'System Design & Support', 
            2:'Computational Health Care Research', 
            3:'',
            4:'Equal Opportunities',
            5:'Analytics & Statistical Modelling',
            6:'Senior Projects',
            7:'ML, Big Data & Cloud Computing',
            8:'Referral Scheme', 
           '-':'-'}

In [17]:
# topics_df1 = pd.DataFrame()
# topics_df2 = pd.DataFrame()
# topics_df3 = pd.DataFrame()
 
# for i, row_list in enumerate(lda_model[corpus]):
#     row = row_list[0] if lda_model.per_word_topics else row_list            
#     row = sorted(row, key=lambda x: (x[1]), reverse=True)
#     for j, (topic_num, prop_topic) in enumerate(row):
#         if len(row) >= 3:        
#             if j ==0:
#                 topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
#             elif j ==1:
#                 topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
#             elif j ==2:
#                 topics_df3 = topics_df3.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
#             else:
#                 break
#         elif len(row) == 2:
#             if j ==0:
#                 topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
#             elif j ==1:
#                 topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
#                 topics_df3 = topics_df3.append(pd.Series(['-', '-']), ignore_index=True)
#         elif len(row) == 1:
#                 topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
#                 topics_df2 = topics_df2.append(pd.Series(['-', '-']), ignore_index=True)  
#                 topics_df3 = topics_df3.append(pd.Series(['-', '-']), ignore_index=True)        
                
            
# topics_df1.rename(columns={0:'1st Topic', 1:'1st Topic Contribution'}, inplace=True)
# topics_df2.rename(columns={0:'2nd Topic', 1:'2nd Topic Contribution'}, inplace=True)
# topics_df3.rename(columns={0:'3rd Topic', 1:'3rd Topic Contribution'}, inplace=True)

# topics_comb = pd.concat([topics_df1, topics_df2, topics_df3],  axis=1, sort=False)

# #Join topics dataframe to original data
# df = pd.concat([df, topics_comb], axis=1, sort=False)

# #df.head(10)

# # #Combine the top 3 into one column so it's easier to analyse
# # df['Top 3 Topics'] = df['1st Topic'].apply(lambda x: lda_dict[x].join(" ,") + " ").astype(str) + df['2nd Topic'].apply(lambda x: lda_dict[x].join(" ,") + " ").astype(str) + \
# # df['3rd Topic'].apply(lambda x: lda_dict[x]).astype(str)

In [18]:
#pip install gensim

In [19]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim_models

In [20]:
#import pyLDAvis.gensim
# import pickle 
# import pyLDAvis
# from pyLDAvis import gensim
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wendionwuakpa/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wendionwuakpa/opt/anaconda3/lib/pytho

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.004578 -0.085662       1        1  30.153609
7     -0.026042 -0.101694       2        1  15.996255
8     -0.043260 -0.068920       3        1  11.953800
9      0.164364  0.076220       4        1   9.492769
2     -0.008142  0.067131       5        1   8.735756
6     -0.050267  0.198096       6        1   7.531361
5      0.034287 -0.080952       7        1   5.898939
1      0.162291  0.076664       8        1   3.811800
3      0.018900 -0.141619       9        1   3.308039
4     -0.247554  0.060735      10        1   3.117671, topic_info=            Term          Freq         Total Category  logprob  loglift
317       status   7260.000000   7260.000000  Default  30.0000  30.0000
580     research  11117.000000  11117.000000  Default  29.0000  29.0000
1448         aws   3018.000000   3018.000000  Default  28.0000  28.0000
212   disability   5285.000000   5285.000000  Default  27.0000  27.0000
234       gender   3944.000000   3944.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
634         deep    421.876952   5091.068084  Topic10  -5.2593   0.9776
20        career    407.777798   4004.315481  Topic10  -5.2933   1.1837
238         help    404.732774   7559.152411  Topic10  -5.3008   0.5408
23        client    380.172433   8576.156298  Topic10  -5.3634   0.3520
271     national    374.520898   4747.080008  Topic10  -5.3784   0.9284

[852 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
21537      8  0.995384       &
3586       2  0.126736       )
3586       3  0.715610       )
3586       5  0.002560       )
3586       6  0.084491       )
...      ...       ...     ...
3816       5  0.024449   youve
3816       7  0.040749   youve
12366      4  0.991703  zillow
29200      6  0.987290  zoetis
12898      6  0.994653  zulily

[2672 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 9, 10, 3, 7, 6, 2, 4, 5])

# Topic Modeling on Khoury Classes

In [7]:
df_courses = pd.read_csv('cs_ds_course_descriptions.csv')
df_courses.head(10)

,course number,titles,descriptions,credits
0,CS 1100,Computer Science and Its Applications,Introduces students to the field of computer s...,(4 Hours)
1,CS 1101,Lab for CS 1100,Accompanies CS 1100. Involves experiments and ...,(1 Hour)
2,CS 1200,First Year Seminar,Seeks to support students in their transition ...,(1 Hour)
3,CS 1210,Professional Development for Khoury Co-op,Continues the preparation of students for care...,(1 Hour)
4,CS 1800,Discrete Structures,Introduces the mathematical structures and met...,(4 Hours)
5,CS 1802,Seminar for CS 1800,Accompanies CS 1800. Illustrates topics from t...,(1 Hour)
6,CS 1990,Elective,Offers elective credit for courses taken at ot...,(1-4 Hours)
7,CS 2500,Fundamentals of Computer Science 1,Introduces the fundamental ideas of computing ...,(4 Hours)
8,CS 2501,Lab for CS 2500,Accompanies CS 2500. Covers topics from the co...,(1 Hour)
9,CS 2510,Fundamentals of Computer Science 2,Continues CS 2500. Examines object-oriented pr...,(4 Hours)


In [8]:
#df_courses = df_courses.drop('Unnamed:0')

In [39]:
# Classes Wordcloud
os.chdir(sys.path[0])

# Read text
text = df_courses['descriptions'].sum()
stopwords = STOPWORDS
additional = {'including', 'students', 'Covers', 'offers', 'opportunity'}
stopwords = stopwords.update(additional)

wc = WordCloud(
    background_color='white',
    stopwords=stopwords,
    height=200,
    width=400
)

wc.generate(text)

#store to file
wc.to_file('wordcloud_output.png')
print(stopwords)

None
